In [34]:
from tools import ova_combo
from tools import aifdb_fetcher
import pandas as pd 
import re 
import os
import json
import urllib
import csv
import wget
import importlib
import requests
from glob import glob

In [57]:
importlib.reload(ova_combo)

<module 'tools.ova_combo' from '/Users/eimear/Documents/Paper_writing/ArgDiffs/code/tools/ova_combo.py'>

Some of the nodesets reported to be part of the corpus have no OVA files. Some of the texts have no timestamps in them.

# Download QT30 OVA by episode

In [ ]:
url = f"http://corpora.aifdb.org/nodesets.php?shortname=qt30"
with urllib.request.urlopen(url) as url:
    nodesets = json.loads(url.read().decode())

In [35]:
with open('../data/reference/qt30_shortcorps.txt', 'r') as file:
        qt30_episode_list = file.read().splitlines()

In [10]:
with open('../data/qt_ep_true_shortnames.csv', 'r', encoding='utf-8-sig') as f:
    qt30_ep_tuples = [tuple(line) for line in csv.reader(f)]
ep_names = [t[1] for t in qt30_ep_tuples]

Check the nodes that are listed for each shortname in the list

In [46]:
base_dir = "../data/orig_episode_maps"
map_type = 'ova'

for shortname, epname in qt30_ep_tuples:
    dir_out = f"{base_dir}/{epname}"
    #os.path.join(base_dir, epname)

    if not os.path.exists(dir_out):
        os.makedirs(dir_out)


    url = f"http://corpora.aifdb.org/nodesets.php?shortname={shortname}"
    with urllib.request.urlopen(url) as url:
        data = json.loads(url.read().decode())
    map_list = data['nodeSets']
    print(f"\nNodesets for {epname}:", data['nodeSets'])
    missing = []
    
    for i, n in enumerate(data['nodeSets']):
        # print("----")
        # print(f"Checking for argmap at nodeset {n} (node {i+1}/{len(data['nodeSets'])})")
        filepath = f"{dir_out}/{str(n)}.json" #os.path.join(dir_out, f"{str(n)}.json")

        # print(f"Checking for file {filepath}")
        if os.path.exists(filepath):
            pass
            # print("Already downloaded!")
        else:
            # print("File doesn't exist yet.")
            # fileurl = url_base + n
            if map_type == 'ova':
                fileurl = f"http://ova.arg.tech/db/{n}"
                # print(f"\nGetting OVA-friendly file", end='')
            else:
                fileurl = f"http://www.aifdb.org/json/{n}"
                # print(f"\nGetting json file at {fileurl}",  end='')

            # print(f" Getting url {fileurl}... ")
            try:
                wget.download(fileurl, filepath)
                # print("\nDone")
            except urllib.error.HTTPError:
                missing.append(n)
                print(f"404: Nodeset {n} appears not to exist")
print("----")
# print(f"Finished with the argument maps in {epname}")
print("==========")


Nodesets for 11november2021: [23460, 23473, 23475, 23476, 23478, 23479, 23480, 23481, 23483, 23484, 23485, 23487, 23488, 23491, 23494, 23495, 23496, 23497, 23498, 23500, 23502, 23503, 23504, 23505, 23506, 23507, 23508, 23509, 23510, 23511, 23512, 23513, 23514, 23515, 23517, 23555, 23688, 24042, 25475, 25409, 25690, 25691]
404: Nodeset 25475 appears not to exist

Nodesets for 28october2021: [23521, 23523, 23525, 23526, 23527, 23530, 23531, 23533, 23535, 23536, 23537, 23538, 23539, 23541, 23542, 23544, 23545, 23547, 23548, 23549, 23550, 23551, 23552, 23553, 23556, 23557, 23558, 23559, 23560, 23561, 23562, 23563, 23565, 23566, 23567, 23569, 23570, 23571, 23572, 23573, 23574, 23575, 23576, 23577, 23578, 23579, 23580, 23581, 23582, 23583, 23584, 23585, 23586, 23587, 23588, 25379, 25474, 23568, 25692]
404: Nodeset 25474 appears not to exist

Nodesets for 14october2021: [23589, 23590, 23591, 23593, 23594, 23595, 23596, 23597, 23599, 23600, 23601, 23602, 23603, 23604, 23605, 23606, 23607, 236

Reporting on missing OVA files

In [32]:
missing_nodesets = pd.DataFrame(columns=['shortname', 'episode', 'missing ova nodeset'])

In [33]:
for shortname, epname in qt30_ep_tuples:
    url = f"http://corpora.aifdb.org/nodesets.php?shortname={shortname}"
    with urllib.request.urlopen(url) as url:
        data = json.loads(url.read().decode())
    map_list = data['nodeSets']
    print(f"Nodesets for {epname}:", data['nodeSets'])

    for n in data['nodeSets']:
        fileurl = f"http://ova.arg.tech/db/{n}"
        r = requests.head(fileurl)
        if r.status_code != 200:
            missing_nodesets = missing_nodesets._append({'shortname': shortname, 'episode': epname, 'missing ova nodeset': n}, ignore_index=True)


Nodesets for 11november2021: [23460, 23473, 23475, 23476, 23478, 23479, 23480, 23481, 23483, 23484, 23485, 23487, 23488, 23491, 23494, 23495, 23496, 23497, 23498, 23500, 23502, 23503, 23504, 23505, 23506, 23507, 23508, 23509, 23510, 23511, 23512, 23513, 23514, 23515, 23517, 23555, 23688, 24042, 25475, 25409, 25690, 25691]
Nodesets for 28october2021: [23521, 23523, 23525, 23526, 23527, 23530, 23531, 23533, 23535, 23536, 23537, 23538, 23539, 23541, 23542, 23544, 23545, 23547, 23548, 23549, 23550, 23551, 23552, 23553, 23556, 23557, 23558, 23559, 23560, 23561, 23562, 23563, 23565, 23566, 23567, 23569, 23570, 23571, 23572, 23573, 23574, 23575, 23576, 23577, 23578, 23579, 23580, 23581, 23582, 23583, 23584, 23585, 23586, 23587, 23588, 25379, 25474, 23568, 25692]
Nodesets for 14october2021: [23589, 23590, 23591, 23593, 23594, 23595, 23596, 23597, 23599, 23600, 23601, 23602, 23603, 23604, 23605, 23606, 23607, 23608, 23609, 23610, 23611, 23612, 23613, 23614, 23615, 23616, 23617, 23618, 23620, 23

In [31]:
missing_nodesets

,shortname,episode,missing ova nodeset
0,qt11112021wtxt,11november2021,25475
1,qt28102021wtxt,28october2021,25474
2,qt14102021wtxt,14october2021,25472
3,qt30092021wtxt,30september2021,25045
4,qt16092021wtxt,16september2021,25473
5,qt02092021wtxt,2september2021,25468
6,qt19082021wtxt,19august2021,25465
7,qt05082021wtxt,5august2021,24992
8,qt22072021wtxt,22july2021,25463
9,cutietestrun8july2021,8july2021,25462


Quite a few appear to have no OVA file...

# Data Cleaning

Fix names

In [36]:
def qt30_text_namefix(text_to_fix, filename):
    for fix_it in [('Luke McCroy Jones', 'Luke McCroy-Jones'), ('Kate Frobes', 'Kate Forbes'),
                   ('Chris Philip', 'Chris Philp'), ('Goerge Mpanga', 'George Mpanga')]:
        if fix_it[0] in text_to_fix:
            print(f"{os.path.basename(filename)}: {fix_it[0]} error")
            text_to_fix = text_to_fix.replace(fix_it[0], fix_it[1])
    return text_to_fix

In [41]:
for ep in ep_names:
    ova_combo.ova_all_ova3(f'../data/orig_episode_maps/{ep}', f'../data/ova3_episode_maps/{ep}')

../data/ova3_episode_maps/11november2021/23515.json
../data/ova3_episode_maps/11november2021/23503.json
../data/ova3_episode_maps/11november2021/23485.json
../data/ova3_episode_maps/11november2021/23688.json
../data/ova3_episode_maps/11november2021/23488.json
../data/ova3_episode_maps/11november2021/23484.json
../data/ova3_episode_maps/11november2021/23555.json
../data/ova3_episode_maps/11november2021/23502.json
../data/ova3_episode_maps/11november2021/23514.json
../data/ova3_episode_maps/11november2021/23509.json
../data/ova3_episode_maps/11november2021/23460.json
../data/ova3_episode_maps/11november2021/23476.json
../data/ova3_episode_maps/11november2021/23513.json
../data/ova3_episode_maps/11november2021/23505.json
../data/ova3_episode_maps/11november2021/25691.json
already OVA3
../data/ova3_episode_maps/11november2021/23483.json
../data/ova3_episode_maps/11november2021/23495.json
../data/ova3_episode_maps/11november2021/23494.json
../data/ova3_episode_maps/11november2021/25690.json

In [42]:
for ep in ep_names:
    print(f'\n*** {ep} ***')
    ep_files = glob(f'../data/ova3_episode_maps/{ep}/*.json')
    # print(ep_files)
    for mapfile in ep_files:
        # print('\t', mapfile)

        with open(mapfile, 'r') as f:
            argmap = json.loads(f.read())
        
        for n in argmap['AIF']['nodes']:
            n['text'] = qt30_text_namefix(n['text'], mapfile)
        argmap['text'] = qt30_text_namefix(argmap['text'], mapfile)

        with open(mapfile, 'w') as f:
            json.dump(argmap, f, indent=4)


*** 11november2021 ***

*** 28october2021 ***
23581.json: Kate Frobes error
23581.json: Kate Frobes error
23581.json: Kate Frobes error
23581.json: Kate Frobes error
23581.json: Kate Frobes error
23581.json: Kate Frobes error
23581.json: Kate Frobes error
23581.json: Kate Frobes error
23581.json: Kate Frobes error
23581.json: Kate Frobes error
23581.json: Kate Frobes error
23581.json: Kate Frobes error
23581.json: Kate Frobes error
23581.json: Kate Frobes error
23581.json: Kate Frobes error
23581.json: Kate Frobes error
23581.json: Kate Frobes error
23581.json: Kate Frobes error
23581.json: Kate Frobes error
23581.json: Kate Frobes error
23581.json: Kate Frobes error
23581.json: Kate Frobes error
23558.json: Kate Frobes error
23558.json: Kate Frobes error
23558.json: Kate Frobes error
23558.json: Kate Frobes error
23558.json: Kate Frobes error
23558.json: Kate Frobes error
23558.json: Kate Frobes error
23558.json: Kate Frobes error
23558.json: Kate Frobes error
23558.json: Kate Frobes

Test first

In [43]:
from analytics import renewed_ova3_toolbox as ova3

ova3.remove_all_meta(argmap, verbose=True)

In [45]:
with open("../data/ova3_episode_maps/2september2020/18841.json", 'r') as f:
    argmap = json.loads(f.read())
ova3.remove_all_meta(argmap, verbose=True)

*** Removing extra nodes and edges ***
Found 12 invisible nodes


Old counts:
	Nodes:  69
	Edges:  87

New counts:
	Nodes:  57
	Edges:  87


{'AIF': {'nodes': [{'nodeID': '1_18841',
    'text': 'Jed Mercurio : we all want to pay tribute to Marcus Rashford for the campaign',
    'type': 'L'},
   {'nodeID': '2_18841', 'text': 'Asserting', 'type': 'YA'},
   {'nodeID': '3_18841',
    'text': 'we all want to pay tribute to Marcus Rashford for the campaign',
    'type': 'I'},
   {'nodeID': '6_18841',
    'text': 'Jed Mercurio : what a great example this young man is as well as being a fantastic footballer',
    'type': 'L'},
   {'nodeID': '7_18841', 'text': 'Asserting', 'type': 'YA'},
   {'nodeID': '8_18841',
    'text': 'what a great example Marcus Rashford is as well as being a fantastic footballer',
    'type': 'I'},
   {'nodeID': '12_18841', 'text': 'Default Transition', 'type': 'TA'},
   {'nodeID': '14_18841', 'text': 'Default Rephrase', 'type': 'MA'},
   {'nodeID': '16_18841', 'text': 'Restating', 'type': 'YA'},
   {'nodeID': '17_18841',
    'text': 'Jed Mercurio : I sometimes despair of hearing things called U‑turns',
    

# Make per-episode OVA

In [58]:
for ep in ep_names:
    ova_combo.combine_ova3(f'../data/ova3_episode_maps/{ep}', f'../data/merged_episodes/{ep}.json', verbose=True)

Combining files in the directory  ../data/ova3_episode_maps/11november2021
41 files found.
Stamp list:  [('[00:51:44]', 0), ('[00:50:18]', 1), ('[00:17:54]', 2), ('[00:15:16]', 3), ('[00:39:14]', 4), ('[00:33:08]', 5), ('[00:11:24]', 6), ('[00:46:58]', 7), ('[01:04:40]', 8), ('[00:48:26]', 9), ('[00:10:36]', 10), ('[00:14:06]', 11), ('[01:01:10]', 12), ('[00:36:08]', 13), ('[00:59:22]', 14), ('[00:31:38]', 15), ('[00:30:20]', 16), ('[00:42:32]', 17), ('[00:56:04]', 18), ('[00:54:36]', 19), ('[00:34:50]', 20), ('[01:03:12]', 21), ('[00:11:56]', 22), ('[00:44:06]', 23), ('[00:53:14]', 24), ('[00:41:02]', 25), ('[00:20:56]', 26), ('[00:26:56]', 27), ('[00:57:34]', 28), ('[00:23:50]', 29), ('[01:06:36]', 30), ('[01:05:38]', 31), ('[00:22:20]', 32), ('[00:19:28]', 33), ('[00:28:46]', 34), ('[00:13:34]', 35), ('[00:16:20]', 36), ('[00:25:30]', 37), ('[00:45:20]', 38), ('[00:12:46]', 39), ('[00:37:48]', 40)]
Sorted stamp list:  [('[00:10:36]', 10), ('[00:11:24]', 6), ('[00:11:56]', 22), ('[00